# 0802字段查询

查询集两大特性：  

    惰性执行：创建查询集不会访问数据库，直到调用数据时，才会访问数据库，  
                调用数据的情况包括迭代、序列化、与if合用  
    缓存：查询集的结果被存下来之后，再次查询时会使用之前缓存的数据  

返回列表的过滤器如下：

    all()：返回所有数据  
    filter()：返回满足条件的数据  
    exclude()：返回满足条件之外的数据，相当于sql语句中where部分的not关键字  
    order_by()：排序  

返回单个值的过滤器如下：

    get()：返回单个满足条件的对象  
> 如果未找到会引发"模型类.DoesNotExist"异常  
> 如果多条被返回，会引发"模型类.MultipleObjectsReturned"异常  

    count()：返回当前查询的总条数  
    aggregate()：聚合  
    exists()：判断查询集中是否有数据，如果有则返回True，没有则返回False  

查询语法：

    说明：属性名称和比较运算符间使用两个下划线，所以属性名不能包括多个下划线
    属性名称__比较运算符=值

> 通过“属性名_id”表示外键对应对象的id值

> get不能for循环


## all()：查询所有数据

例：查询所有图书信息
```python
list = BookInfo.objects.all()
```

## 条件运算符



### exact：判等

例：查询编号为1的图书信息  

```python
list = BookInfo.objects.filter(id__exact=1)
```
    
  可简写：
```python
list = BookInfo.objects.filter(id=1)
```
     

### contains：是否包含

如果要包含%无需转义，直接写即可  

例：查询书名包含“传”的图书信息

```python
list = BookInfo.objects.filter(btitle__contains='传
```

### startswith & endswith：以指定值开头/结尾

例：查询书名以“部”结尾的图书信息
```python
list = BookInfo.objects.filter(btitle__endswith='部')
```

例：查询书名以“天”开头的图书信息
```python
list = BookInfo.objects.filter(btitle__startswith='天')
```

> 以上运算符均区分大小写，在运算符前加上i则表示不区分大小写  
> 
> 如：iexact、icontains、istartswith、iendswith

### in：是否包含在某范围

例：查询标号为1或3或5的图书信息
```python 
list = BookInfo.objects.filter(pk__in=[1, 3, 5])
```


### isnull：是否为null(空值)

例：查询书名不为空的图书信息
```python
list = BookInfo.objects.filter(btitle__isnull=False)
```

### gt--大于 & gte--大于等于 & lt--小于 & lte--小于等于

例：查询编号大于3的图书
```python 
list = BookInfo.objects.filter(id__gt=3)
```

### exclude：不等于过滤器

在判断不等于条件时，使用exclude()过滤器排除等于条件  

例：查询编号不等于3的图书信息
```python 
list = BookInfo.objects.filter(id__gt=3)
```

### 对日期时间类型进行查询

比较运算符使用year、month、day、week_day、hour、minute、second 对日期类型的属性进行查询判断  

例：查询1980年发表的图书信息
```python 
list = BookInfo.objects.filter(bpub_date__year=1980)
```

例：查询1980年1月1日后发表的图书信息
```python
from datetime import date
list = BookInfo.objects.filter(bpub_date__gt=date(1990, 1, 1))
```
> 键入日期时间需要调用datetime模块

### range：区间查询

`__range=(a,b)`用于查询a-b之间的所有数据，要与in区分。  
> in按范围查询，输入列表，仅请求列表内相匹配的数据  
> range按区间查询，如果元组，请求元组区间内的所有数据  

例：查询阅读量在10-30之间的图书信息
```python
list = BookInfo.objects.filter(bcommet__range=(10, 30))
```

### regex：正则查询

`__regex=正则表达式`用于按照等号后的正则表达式规则进行查询  

例：利用正则查询书名开头为“zh”或“The”一个或多个的图书信息
```python 
list = BookInfo.objects.get(btitle__regex=r'^(zh|The)+')
```

> iregex不区分大小写的正则查询

## 获取查询集第一个——first()

`.first()`函数返回结果集的第一个对象，当没有时返回`None`。  

例：获取第一个图书信息
```python
list = BookInfo.objects.all().first()
```


## 获取查询集最后一个——last()

与`.first()`用法相同

例：获取最后一个图书信息
```python
list = BookInfo.objects.all().last()
```


## 判断是否存在——exists()

`.exists()`判断是否存在，如果存在返回`True`，不存在返回`False`.

例：判断是否含有编号为6的图书
```python 
list = BookInfo.objects.filter(id=6).exists()
```

## 关联查询

含有外键的查询，类似于join查询，可以通过外键的数据表查询本数据表信息，也可通过本表查询与之关联的外键表的信息。    

语法：  

    关联模型类名小写__属性名__运算符=值
    

例：查询图书，要求图书中英雄的描述包含“八”字的 

```python 
list = BookInfo.objects.filter(heroinfo__hcontent__contains='八')
```

例：查询书名为“天龙八部”的所有英雄
```python
list = HeroInfo.objects.filter(hbook__btitle='天龙八部')
```

## F对象——属性间比较

之前使用的查询都是属性与常量的比较，但是没办法实现两个属性之间的比较  
django给出了一个特殊的对象可以时间属性间比较的功能，该对象称为——F对象  

导入F对象，F对象被定义在`django.db.models`中
```python 
from django.db.models import F
```
例：查询阅读量大于等于评论量的图书信息

```python 
list = BookInfo.objects.filter(bread__gte=F('bcommet'))
```

F对象上可以进行算术运算

例：查询阅读量大于2倍评论量的图书信息
```python
list = BookInfo.objects.filter(bread__gt=F('bcommet') * 2)
```


## Q对象

多个过滤器逐个调用表示逻辑与关系，同SQL语句中where部分的and关键字，用到Q对象  

导入Q对象，Q对象被定义在`django.db.models`中

from django.db.models import Q

例：查询阅读量大于20，并且编号小于3的图书信息
```python 
list = BookInfo.objects.filter(bread__gt=20,id__lt=3)
```
或
```python
list = BookInfo.books.filter(bread__gt=20).filter(id__lt=3)
```
如果用Q对象，需和`|`管道符配合使用实现逻辑或or的查询
```python
list = BookInfo.objects.filter(Q(bread__gt=20) | Q(id__lt=3))
```

> Q对象可以使用&、|链接
> 
> &表示逻辑与
> |表示逻辑或

Q对象前，可以使用`~`操作符，表示not非

例：查询标号不等于3的图书
```python
list = BookInfo.objects.filter(~Q(pk=3))
```



## 排序——order_by()

order_by()自定义排序，默认升序排序，在条件前加`-`以实现倒序排序

```python
list = BookInfo.objects.all().order_by('-bread')  # 倒序排序

list = BookInfo.objects.all().order_by('bread')   # 升序排序
```



## 修改数据——update()

例：将所有图书的阅读量更改为3
```python
BookInfo.objects.all().update(bread='3')
```

例：将编号为3的图书的阅读量更改为50
```python
BookInfo.objects.filter(id=3).update(bread='50')
```


## 删除数据——delete()

例：删除阅读量等于50的图书信息
```python 
BookInfo.objects.filter(bread='50').delete()
```


## 聚合函数

聚合函数包括：Avg、Count、Max、Min、Sum，均被定义在`django.db.models`中  
使用之前一定要先导入相应模块  

使用`aggregate()`过滤器来调用聚合函数  

例：查询图书的总阅读量  
```python
list = BookInfo.objects.aggregate(Sum('bread'))
```

> count数据统计无需导入模块可以直接使用
```python
list = BookInfo.objects.count()
```
